In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.schema import BaseOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate

set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1
)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

# 리스트를 쉼표로 구분된 문자열로 변환하는 함수
def convert_list_to_string(activity_list):
    return ', '.join(activity_list)


template = ChatPromptTemplate.from_messages(
    [
        ("system", "해외 여행 시 필요한 짐 목록을 자세한 체크리스트 형태로 알려주세요. 체크리스트 항목은 최소 20개의 목록으로 답해야 합니다. 한국어로만 답변해 주세요."),
        ("human", "저는 남성이고, 아기와 함께 서울로 여행을 갑니다. 5월 10일에 출발하여 5일간 머물 예정입니다. 여행 중 주로 수영, 등산을 즐길 계획입니다. 여행 시 중요하게 생각하는 것은 숙면입니다. 위 사항을 참고하여 가능한 자세한 체크리스트 항목을 만들어주세요."),
        ("ai",  
         "필수 항목: 여권 및 비자, 항공권, 숙박 예약 확인서, 국제 운전 면허증, 여행자 보험, 현금 및 신용카드/체크카드, 휴대폰 및 충전기 \
          기본 항목: 재킷, 립밤, 비타민, 약, 진통제, 시계, 열쇠, 속옷, 긴 양말, 짧은 양말, 반바지, 캐쥬얼 바지, 벨트, 잠옷, 안경, 귀마개, 안대, 책, 카메라, 우산, 스웨터  \
          세면 도구: 콘택트 렌즈, 렌즈 세척액, 손톱깍이, 핀셋, 칫솔, 치약, 빗, 치실, 향수, 면도기, 쉐이빙젤, 면봉   \
          수영: 수영복, 선글라스, 자외선 차단제, 모자, 샌들, 타월 \
          등산: GPS, 등산 셔츠, 등산 바지, 등산 양말, 선글라스, 물병, 모자, 벌레 퇴치 스프레이, 구급상자, 나침반, 등산화 \
          아기: 아기 물병 \
        "),
        # 여기서부터 실제 사용자의 입력을 처리합니다.
        (
            "human",
            "저는 {gender}이고, {who}와 함께 {destination}으로 해외 여행을 갈 예정입니다. \
            {start_month}월 {start_day}일에 출발해서 {days}일 동안 머물 예정입니다. \
            여행 중에는 {activities}를 즐길 계획입니다. \
            또한, 여행 시 중요하게 생각하는 것은 {considers}입니다. \
            위 사항을 참고하여 가능한 자세한 체크리스트 항목을 만들어주세요."
        ),
    ]
)

activities_list = ["수영", "등산", "자전거"]
who_list = ["부모님"]
considers_list = ["숙면"]

input_data = {
    "gender": "man",
    "destination": "Seoul",
    "start_month": "5",
    "start_day": "10",
    "days": "5",
    "activities": convert_list_to_string(activities_list),
    "who" : convert_list_to_string(who_list),
    "considers" : convert_list_to_string(considers_list)
}

chain = template | chat #| CommaOutputParser()
# Invoke the chain with the input
output = chain.invoke(input_data)

# Print the output
print(output)



content='필수 항목: 여권 및 비자, 항공권, 숙박 예약 확인서, 국제 운전 면허증, 여행자 보험, 현금 및 신용카드/체크카드, 휴대폰 및 충전기\n의류: 티셔츠, 셔츠, 반바지, 바지, 양말, 속옷, 수영복, 등산복, 자전거복, 방한용품(가벼운 외투, 비옷), 모자, 슬리퍼\n화장품/세면도구: 칫솔, 치약, 샴푸, 바디워시, 선크림, 립밤, 화장품, 면도기, 면도크림, 헤어브러쉬, 타올\n액세서리: 선글라스, 모자, 시계, 지갑, 목걸이, 팔찌, 반지\n의료용품: 약(감기약, 소화제, 진통제 등), 상처소독약, 밴드, 면봉, 붕대, 해열제, 구급약통\n기타: 여행용 가방, 스마트폰, 노트북 또는 태블릿, 여행용 어플(지도, 번역 등), 여행 일정 및 지도, 여행용 우산, 스니커즈, 스포츠화, 스포츠용품(수영용품, 등산용품, 자전거 장비), 사진기, 충전기 및 어댑터, 여행용 보조배터리, 책 또는 이어폰, 스케치북 및 필기도구, 음식물 및 물병, 여행용 건전지 및 소독제, 여행용 습기제거제, 숙면을 위한 이어플러그 및 아이 마스크'
